### Extract Embeddings

Based on https://github.com/rom1504/clip-retrieval

In [1]:
import os
import torch
import clip
from PIL import Image
from tqdm import tqdm
import numpy as np

/opt/anaconda3/envs/resnet/lib/python3.10/site-packages/clip/clip.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging


#### iNaturalist Embeddings

In [2]:
# Settings
IMG_DIR = "images"
OUT_DIR = "inat_emb"
os.makedirs(OUT_DIR, exist_ok=True)

In [3]:
# Load model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
# Get image paths
valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.webp')
image_paths = sorted([os.path.join(IMG_DIR, f) for f in os.listdir(IMG_DIR) 
                      if f.lower().endswith(valid_exts)])

print(f"Processing {len(image_paths)} images on {device}...")

In [4]:
g

NameError: name 'g' is not defined

In [4]:
# Extraction loop
embeddings = []
metadata = []

for path in tqdm(image_paths):
    try:
        image = preprocess(Image.open(path)).unsqueeze(0).to(device)
        
        with torch.no_grad():
            features = model.encode_image(image)
            features /= features.norm(dim=-1, keepdim=True)    # normalize for cosine similarity
            
        embeddings.append(features.cpu().numpy())
        metadata.append(path)
        
    except Exception as e:
        print(f"Skipping corrupt image {path}: {e}")

NameError: name 'image_paths' is not defined

In [ ]:
if embeddings:
    emb_matrix = np.vstack(embeddings)
    
    np.save(os.path.join(OUT_DIR, "embeddings.npy"), emb_matrix)
    
    with open(os.path.join(OUT_DIR, "filenames.txt"), "w") as f:
        f.write("\n".join(metadata))
        
    print(f"Success! Saved {emb_matrix.shape} matrix to {OUT_DIR}/embeddings.npy")

Processing 1238 images on cpu...


  4%|▍         | 48/1238 [00:09<04:00,  4.95it/s]


KeyboardInterrupt: 